In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds, basinhopping

In [ ]:
# Physical constraints
E = 1e4  # Relative tensile modululs
t = 1.2  # Spring thickness

n = 20  # Number of spring elements to compute
keep_out_penalty = 1e6

# Geometry restrictions from the robot
# Coordinate system based on the axis of the middle gear being the origin
neutral_roller_center = np.array([17.58, -7.2])
deflected_roller_center = np.array([18.62, 3.78])
shaft_center = np.array([0, 0])

roller_radius = 8
shaft_radius = 5.5

rolling_end_neutral_angle = 13.58 * np.pi / 180
rolling_end_deflected_angle = 89 * np.pi / 180

In [ ]:
def keep_out_check(r, origin, radius, endpoints=False):
    if endpoints:
        return np.sum((r - origin)**2, axis=1) < radius*radius
    else:
        return np.sum((r[1:-1] - origin)**2, axis=1) < radius*radius


# Define the end points of the spring
end_point_ds = 1

fixed_end = np.array([1, -9.56])
rolling_end_neutral = np.array([[13.7, 1],
                                [13.7 + end_point_ds*np.cos(rolling_end_neutral_angle), 1 + end_point_ds*np.sin(rolling_end_neutral_angle)]])
rolling_end_deflected = np.array([[9.7, 2.1],
                                  [9.7 + end_point_ds*np.cos(rolling_end_deflected_angle), 2.1 + end_point_ds*np.sin(rolling_end_deflected_angle)]])

In [ ]:
#  Neutral spring postion initial guess
guess_r0 = np.zeros([n, 2])
guess_r0[:, 0] = np.linspace(1, 13.7, num=n+1, endpoint=False)[1:]
guess_r0[:, 1] = (guess_r0[:, 0] - 13.7) * (10.56 / 12.7) + 1

#  Deflected spring position initial guess
guess_r1 = np.zeros_like(guess_r0)
guess_r1[:, 0] = np.linspace(1, 9.7, num=n+1, endpoint=False)[1:]
guess_r1[:, 1] = (guess_r1[:, 0] - 9.7) * (11.66 / 8.7) + 2.1

#  Initital Guess
x0 = np.zeros(4*n)
x0[:n] = guess_r0[:, 0]
x0[n:2*n] = guess_r0[:, 1]
x0[2*n:3*n] = guess_r1[:, 0]
x0[3*n:4*n] = guess_r1[:, 1]

In [ ]:
#  Plot the initial guess and keep out zones
shaft_circle = plt.Circle(shaft_center, shaft_radius, color='r')
neutral_roller_circle = plt.Circle(neutral_roller_center, roller_radius, color='r')


plt.close()
fig, ax = plt.subplots()
plt.scatter(guess_r0[:, 0], guess_r0[:, 1])
ax.add_patch(shaft_circle)
ax.add_patch(neutral_roller_circle)

plt.scatter(fixed_end[0], fixed_end[1])
plt.scatter(rolling_end_neutral[:, 0], rolling_end_neutral[:, 1])

plt.title("Neutral Spring Initial Guess")
plt.axis('equal')
plt.tight_layout()

plt.show()

In [ ]:
shaft_circle = plt.Circle(shaft_center, shaft_radius, color='r')
deflected_roller_circle = plt.Circle(deflected_roller_center, roller_radius, color='r')

plt.close()
fig, ax = plt.subplots()
plt.scatter(guess_r1[:, 0], guess_r1[:, 1])
ax.add_patch(shaft_circle)
ax.add_patch(deflected_roller_circle)


plt.scatter(fixed_end[0], fixed_end[1])
plt.scatter(rolling_end_deflected[:, 0], rolling_end_deflected[:, 1])

plt.title("Deflected Spring Initial Guess")
plt.axis('equal')
plt.tight_layout()

plt.show()

In [ ]:
# Discretized vector calc functions

def velocity(r):
    # The curve's velocity as a function of the indexing
    return (r[2:] - r[:-2])/2

def acceleration(r):
    # The curve's acceleration as a function of the indexing
    return r[:-2] - 2*r[1:-1] + r[2:]

def cross(v, a):
    # The z component of the cross product that is proportional to the signed curvature
    return (v[:, 0] * a[:, 1]) - (v[:, 1] * a[:, 0])

def arc_length(r):
    return (np.sqrt(np.sum((r[:-2] - r[1:-1])**2, axis=1)) + np.sqrt(np.sum((r[1:-1] - r[2:])**2, axis=1)))/2
    
def curvature(r):
    # r is the position of the spring elements
    s = arc_length(r)
    return cross(velocity(r), acceleration(r)) / s**3

In [ ]:
# Stress functions

def neutral_axis_displacement(k):
    # Returns the distance from the netural axis to the inner edge for a curved beam with curvature k
    # Here it is assumed k>=0
    return np.divide(np.exp(-k*t)*(1+k*t)-1, k*(np.exp(-k*t)-1), where=(k!=0)) + t/2 * (k==0).astype(np.float64)

def flexural_strain(k0, k1, y):
    # Increasing y represents fibers towards the center of curvature
    # k is signed to account for the case where a curved beam is made straight and then curves the other way
    return (k0 - k1) * y / (1 - np.abs(k0)*y)

def tensile_strain(s0, s1):
    return (s1 - s0) / s0

In [ ]:
def stress(r0, r1):
    s0 = arc_length(r0)
    s1 = arc_length(r1)
    k0 = curvature(r0)
    k1 = curvature(r1)
    
    return (np.abs(flexural_strain(k0, k1, neutral_axis_displacement(np.abs(k1)))) +
            E * np.abs(tensile_strain(s0, s1)) +
            keep_out_penalty * (keep_out_check(r0, shaft_center, shaft_radius) +
                                keep_out_check(r1, shaft_center, shaft_radius) +
                                keep_out_check(r0, neutral_roller_center, roller_radius) +
                                keep_out_check(r1, deflected_roller_center, roller_radius)))

def stress_objective_func(x):
    # x is the optimization parameters
    # the first n elements are the x-components of the neutral spring
    # the next n elements are the y-components of the netural spring
    # the next n elements are the x-components of the deflected spring
    # the next n elements are the y-components of the deflected spring    
    r0 = np.zeros([n+3, 2])
    r0[0] = fixed_end
    r0[-2:] = rolling_end_neutral
    r0[1:-2, 0] = x[:n]
    r0[1:-2, 1] = x[n:2*n]
    
    r1 = np.copy(r0)
    r1[-2:] = rolling_end_deflected 
    r1[1:-2, 0] = x[2*n:3*n]
    r1[1:-2, 1] = x[3*n:4*n]

    # TODO choose a much better objective function
    p = 3
    s = stress(r0, r1)
    return np.sum(s**p)**(1/p)

In [ ]:
stress_objective_func(x0)

In [ ]:
# Warning, running this cell will eat all your cpu resources for a bit

bounds = Bounds(([-2]*n + [-10]*n)*2,
                ([15]*n + [3]*n)*2)

minimizer_kwargs = {"method": 'trust-constr', "bounds": bounds}

solution = basinhopping(stress_objective_func, x0, minimizer_kwargs=minimizer_kwargs)
#solution = minimize(stress_objective_func, x0, method='trust-constr', bounds=bounds)

In [ ]:
# TODO plot the results of the optimization
# TODO export the solution into an onshape friendly format, maybe a dxf file

In [ ]:
# Plot the solution

sol_neutral = np.zeros([n+3, 2])
sol_neutral[0] = fixed_end
sol_neutral[-2:] = rolling_end_neutral
sol_neutral[1:-2, 0] = solution.x[:n]
sol_neutral[1:-2, 1] = solution.x[n:2*n]

plt.close()
fig, ax = plt.subplots()
plt.scatter(sol_neutral[:, 0], sol_neutral[:, 1])
plt.plot(sol_neutral[:, 0], sol_neutral[:, 1])
plt.tight_layout()
plt.show()

In [ ]:
sol_deflected = np.zeros([n+3, 2])
sol_deflected[0] = fixed_end
sol_deflected[-2:] = rolling_end_deflected
sol_deflected[1:-2, 0] = solution.x[2*n:3*n]
sol_deflected[1:-2, 1] = solution.x[3*n:4*n]

plt.close()
fig, ax = plt.subplots()
plt.scatter(sol_deflected[1:-1, 0], sol_deflected[1:-1, 1], c=stress(sol_neutral, sol_deflected))
plt.plot(sol_deflected[1:-1, 0], sol_deflected[1:-1, 1])
plt.tight_layout()
plt.show()

# You don't need to run cells past this point these are sanity checks for the functions

In [ ]:
keep_out_test = np.zeros([50, 2])
keep_out_test[:, 0] = np.linspace(0, 10) + neutral_roller_center[0]
keep_out_test[:, 1] = np.linspace(0, 10) + neutral_roller_center[1]

plt.close()
neutral_roller_circle = plt.Circle(neutral_roller_center, roller_radius, color='r')
fig, ax = plt.subplots()
ax.add_patch(neutral_roller_circle)
plt.scatter(keep_out_test[:, 0], keep_out_test[:, 1], 
            c=keep_out_check(keep_out_test, neutral_roller_center, roller_radius, endpoints=True), cmap='Pastel1')
plt.title('Keep Out Penalty Test')
plt.axis('equal')
plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
# Testing the vector calc functions
theta = np.linspace(0, np.pi*2, num=300)
circle_path = np.zeros([theta.shape[0], 2])
circle_path[:, 0] = 3*np.cos(theta)
circle_path[:, 1] = 3*np.sin(theta)

vel_test = velocity(circle_path)
acc_test = acceleration(circle_path)
arc_test = arc_length(circle_path)
crv_test = curvature(circle_path)

In [ ]:
# Test the velocity function
plt.close()
fig, ax = plt.subplots()
plt.quiver(circle_path[1:-1:10, 0], circle_path[1:-1:10, 1], vel_test[::10, 0], vel_test[::10, 1])
plt.axis('equal')
plt.title('Velocity Test')
plt.tight_layout()
plt.show()

In [ ]:
# Test the acceleration function
plt.close()
fig, ax = plt.subplots()
plt.quiver(circle_path[1:-1:10, 0], circle_path[1:-1:10, 1], acc_test[::10, 0], acc_test[::10, 1])
plt.axis('equal')
plt.title('Acceleration Test')
plt.tight_layout()
plt.show()

In [ ]:
# Test the arc length function
print(np.allclose(arc_test, arc_test[0]))
print('Computed circumference: {:.3f}, Analytical circumference: {:.3f}'.format(np.sum(arc_test) + arc_test[0], 
                                                                               2*np.pi*3))

In [ ]:
# Test the curvature function
# Curvature should be the reciprocal of the radius
# and change sign when we trace it in the other direction
print(np.allclose(crv_test, crv_test[0]))
print(1/crv_test[0])
print(1/curvature(circle_path[::-1])[0])

In [ ]:
# Test the stress functions
plt.close()
X = np.linspace(0, 20)
plt.plot(X, neutral_axis_displacement(X))
plt.title('Neutral axis position vs initial curvature')
plt.xlabel('Neutral Axis Position')
plt.ylabel('Initial Curvature')
plt.tight_layout()
plt.show()

In [ ]:
X = np.linspace(-2/t, 2/t, num=200)
Y = np.copy(X)

K0, K1 = np.meshgrid(X, Y)

plt.close()
plt.contourf(K0, K1, np.abs(flexural_strain(K0, K1, neutral_axis_displacement(np.abs(K0)))))
plt.colorbar()
plt.title('Flexural Strain')
plt.xlabel('Neutral Curvature')
plt.ylabel('Deflected Curvature')
plt.axis('scaled')
plt.tight_layout()
plt.show()

In [ ]:
# Tensile strain testing

N = 50
X = np.ones([N, 2])
X[:, 0] = np.linspace(0, 3*np.pi, num=N)

theta = np.linspace(0, np.pi, num=N)
curved_path = np.zeros([X.shape[0], 2])
curved_path[:, 0] = 3*np.cos(theta)
curved_path[:, 1] = 3*np.sin(theta)

tensile_test = tensile_strain(arc_length(X), arc_length(curved_path))
print(np.allclose(tensile_test, tensile_test[0]))
print(tensile_test[0])
# Tensile strain error seems to go to zero as we increase N

In [ ]:
N = 50
X = np.zeros([N, 2])
X[:, 0] = np.linspace(0, 10, num=N)

plt.close()
plt.plot(X[1:-1, 0], tensile_strain(arc_length(X), arc_length(X*X)))
plt.xlabel('X')
plt.ylabel('Tensile Strain')
plt.show()